# OptCache Report Builder

Этот скрипт загружает данные оптимизации из CSV файлов и выводит топ 5 результатов по убыванию custom_fitness и profit.

In [84]:
filter_rule = {
    'fields': {
        'profit': {
            'enabled': True,
            'range': (0.01, float('inf')),
            'color_ranges': []
        },
        'trades_per_30_days': {
            'enabled': True,
            'range': (0.5, float('inf')),
            'color_ranges': []
        },
        'profit_per_30_days_percent': {
            'enabled': False,
            'range': (0.005, float('inf')),
            'color_ranges': [
                {'range': (-float('inf'), 0.010), 'color': 'red'},
                {'range': (0.010, 0.020), 'color': 'orange'},
                {'range': (0.020, float('inf')), 'color': 'green'}
            ]
        },
        'custom_fitness': {
            'enabled': True,
            'range': (0.75, float('inf')),
            'color_ranges': [
                {'range': (0.00, 0.60), 'color': 'red'},
                {'range': (0.60, 0.89), 'color': 'orange'},
                {'range': (0.89, float('inf')), 'color': 'green'}
            ]
        },
        'reldrawdownpercnt_e': {
            'enabled': True,
            'range': (0, 15),
            'color_ranges': [
                {'range': (0.00, 10.0),  'color': 'green'},
                {'range': (10.00, 25.0), 'color': 'orange'},
                {'range': (25.0, float('inf')), 'color': 'red'}
            ]
        },
    },
    'max_runs_count': 5,
    'sort_by': ['profit', 'custom_fitness', 'reldrawdownpercnt_e'],
    'sort_dir': [False, False, True]
}

In [85]:
import pandas as pd
import numpy as np
from pathlib import Path

In [86]:
def BuildReportRows(optcache_filename: str, filter: dict) -> tuple: 

    def BuildHeader(header_df: pd.DataFrame) -> list[str]:
        row = header_df.iloc[0]

        # Format dates as YYYY-MM-DD
        date_from = pd.to_datetime(row["date_from"]).strftime("%Y-%m-%d")
        date_to = pd.to_datetime(row["date_to"]).strftime("%Y-%m-%d")

        report = [f'#### {optcache_filename}',
              '| Expert | Symbol | Interval | Days | TF | Deposit | Lev | Server | Ticks |',
              '| - | - | - | - | - | - | - | - | - |',
              f'| {row["expert_name"]} | ==**{row["symbol"]}**== | [{date_from}; {date_to}) | {row["days"]} | {row["period"]} | {row["trade_deposit"]}{row["trade_currency"]} | {row["trade_leverage"]} | {row["server"]} | {row["ticks_mode"]} | \n'
             ]

        return report        
    
    def colorize(field_name, value, text):
        rule = filter_rule.get('fields', {}).get(field_name, {})

        if not rule:
            return text

        color_ranges = rule.get('color_ranges', [])
        if not color_ranges:
            return text

        for cr in color_ranges:
            if value >= cr['range'][0] and value < cr['range'][1]:
                return f'<span style="background-color:{cr["color"]};">{text}</span>'
        return text

    header_file = Path(f'{optcache_filename}.Header.csv')
    data_file = Path(f'{optcache_filename}.Data.csv')

    if not header_file.exists() or not data_file.exists():
        print(f"Files {header_file} or {data_file} do not exist.")
        return ([], [])

    # add fields to header
    header_df = pd.read_csv(header_file, sep=';', encoding='utf-16')
    header_df['days'] = (pd.to_datetime(header_df['date_to']) - pd.to_datetime(header_df['date_from'])).dt.days

    # add fields to data
    data_df = pd.read_csv(data_file, sep=';', encoding='utf-16')
    data_df['win_rate'] = data_df['profittrades'] / data_df['trades']
    data_df['profit_per_30_days'] = data_df['profit'] / header_df['days'].iloc[0] * 30
    data_df['profit_per_365_days'] = data_df['profit'] / header_df['days'].iloc[0] * 365
    data_df['profit_per_30_days_percent'] = data_df['profit'] / header_df['days'].iloc[0] * 30 / data_df['initial_deposit']
    data_df['profit_per_365_days_percent'] = data_df['profit'] / header_df['days'].iloc[0] * 365 / data_df['initial_deposit']
    data_df['trades_per_30_days'] = data_df['trades'] / header_df['days'].iloc[0] * 30
    data_df.fillna(0, inplace=True)

    # filter data
    report_data = data_df.copy()
    for column, rule in filter.get('fields', {}).items():
        if not rule.get('enabled', True):
            continue
        if not rule.get('range'):
            continue
        min_val, max_val = rule['range']
        report_data = report_data[(report_data[column] >= min_val) & (report_data[column] <= max_val)]

    report_data = report_data.sort_values(filter_rule.get('sort_by', []), ascending=filter_rule.get("sort_dir", []))
    if 'max_runs_count' in filter:
        report_data = report_data.head(filter.get('max_runs_count'))
    # report_data = report_data.reset_index(drop=True)

    data_report = [f'**Top {filter.get("max_runs_count", len(data_df))} runs of {len(data_df)}:**',
                  "| # | Pass | Profit | <span title='Profit per 30 days, %'>30d,%</span> | <span title='Win Rate'>WR</span> | <span title='Custom Criterion'>CC</span> | Trades | <span title='Trades per 30 days'>30d</span> | <span title='Equity Drawdown, %'>Eq. DD</span> | <span title='Profit Factor'>PF</span> | <span title='Recovery Factor'>RF</span> | Sharpe | Rank | Comment |",
                  "|---|------|--------|--------------|----|----|--------|-------|--------|----|----|--------| - | - |"]
    row_template = "| {i} | {pass_num} | {profit} | {profit_per_30_days_percent} | {win_rate} | {custom_fitness} | {trades} | {trades_per_30_days} | {reldrawdownpercnt_e} | {profit_factor} | {recovery_factor} | {sharpe_ratio} | ☆☆☆☆☆ | |"
    
    data_report_cvs  = []
    row_template_cvs = "{i};{filename};{symbol};{period};{date_from};{date_to};{deposit};{leverage};{server};{ticks_mode};{pass_num};{profit};{profit_per_30_days_percent};{profit_per_365_days_percent};{win_rate};{custom_fitness};{trades};{trades_per_30_days};{reldrawdownpercnt_e};{profit_factor};{recovery_factor};{sharpe_ratio}"

    for i, (_, data_row) in enumerate(report_data.iterrows(), 1):
        data_report.append(row_template.format(
            i=int(i),
            pass_num=int(data_row['Pass']),
            symbol=header_df['symbol'].iloc[0],
            tf=header_df['period'].iloc[0],
            interval=header_df['days'].iloc[0],
            profit=colorize('profit', data_row['profit'], round(data_row['profit'], 2)),
            profit_per_30_days_percent=colorize('profit_per_30_days_percent', data_row['profit_per_30_days_percent'],
                                                round(data_row['profit_per_30_days_percent'] * 100, 1)),
            profit_per_365_days_percent=round(data_row['profit_per_365_days_percent'] * 100, 1),
            win_rate=round(data_row['win_rate'] * 100, 1),
            custom_fitness=colorize('custom_fitness', data_row['custom_fitness'], round(data_row['custom_fitness'], 2)),
            trades=data_row['trades'],
            trades_per_30_days=round(data_row['trades_per_30_days'], 1),
            reldrawdownpercnt_e=colorize('reldrawdownpercnt_e', data_row['reldrawdownpercnt_e'],
                                        round(data_row['reldrawdownpercnt_e'], 2)),
            profit_factor=round(data_row['profit_factor'], 2),
            recovery_factor=round(data_row['recovery_factor'], 2),
            sharpe_ratio=round(data_row['sharpe_ratio'], 2)
        ))
        
        data_report_cvs.append(row_template_cvs.format(
            i=int(i),
            filename=optcache_filename,
            symbol=header_df['symbol'].iloc[0],
            period=header_df['period'].iloc[0],
            date_from=header_df['date_from'].iloc[0],
            date_to=header_df['date_to'].iloc[0],
            deposit=header_df['trade_deposit'].iloc[0],
            leverage=header_df['trade_leverage'].iloc[0],
            server=header_df['server'].iloc[0],
            ticks_mode=header_df['ticks_mode'].iloc[0],
            
            pass_num=int(data_row['Pass']),
            profit=data_row['profit'],
            profit_per_30_days_percent=round(data_row['profit_per_30_days_percent'] * 100, 1),
            profit_per_365_days_percent=round(data_row['profit_per_365_days_percent'] * 100, 1),
            win_rate=round(data_row['win_rate'] * 100, 1),
            custom_fitness=round(data_row['custom_fitness'], 2),
            trades=data_row['trades'],
            trades_per_30_days=round(data_row['trades_per_30_days'], 1),
            reldrawdownpercnt_e=round(data_row['reldrawdownpercnt_e'], 2),
            profit_factor=round(data_row['profit_factor'], 2),
            recovery_factor=round(data_row['recovery_factor'], 2),
            sharpe_ratio=round(data_row['sharpe_ratio'], 2)
        ))

    report = BuildHeader(header_df)
    if len(data_report) > 3:
        report += data_report
    else:
        report.append(f"==**No any run meets filter criteria. Total runs {len(data_df)}.**==")

    report.append("\n")
    return (report, data_report_cvs)

## 2. Сортировка и вывод топ 5 результатов

In [87]:
dir = Path('csv')

report_md = []
report_cvs = ["num;filename;symbol;period;date_from;date_to;deposit;leverage;server;ticks_mode;pass_num;profit;profit_per_30_days_percent;profit_per_365_days_percent;win_rate;custom_fitness;trades;trades_per_30_days;reldrawdownpercnt_e;profit_factor;recovery_factor;sharpe_ratio"]
for opt_file in dir.glob("*.opt"):
    (md, cvs) = BuildReportRows(opt_file, filter_rule)
    report_md += md
    report_cvs += cvs

with open("optcache_report.md", "w", encoding="utf-8") as f:
    for line in report_md:
        f.write(line + "\n")

with open("optcache_report.csv", "w", encoding="utf-8") as f:
    for line in report_cvs:
        f.write(line + "\n")